# Hiperparámetros en árboles de decisión

In [ ]:
# cargamos los conjuntos de datos de clasificación y regresión.

import pandas as pd

data_clf_columns = ["Culmen Length (mm)", "Culmen Depth (mm)"]
target_clf_column = "Species"
data_clf = pd.read_csv("../../data/penguins/penguins_classification.csv")

data_reg_columns = ["Flipper Length (mm)"]
target_reg_column = "Body Mass (g)"
data_reg = pd.read_csv("../../data/penguins/penguins_regression.csv")

# Crear funciones auxiliares
Crearemos algunas funciones auxiliares para graficar los datos, así como el límite de decisión (para la clasificación) y la línea de regresión (para la regresión).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import DecisionBoundaryDisplay


def fit_and_plot_classification(model, data, feature_names, target_names):
    model.fit(data[feature_names], data[target_names])
    if data[target_names].nunique() == 2:
        palette = ["tab:red", "tab:blue"]
    else:
        palette = ["tab:red", "tab:blue", "black"]
    DecisionBoundaryDisplay.from_estimator(
        model, data[feature_names], response_method="predict",
        cmap="RdBu", alpha=0.5
    )
    sns.scatterplot(data=data, x=feature_names[0], y=feature_names[1],
                    hue=target_names, palette=palette)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


def fit_and_plot_regression(model, data, feature_names, target_names):
    model.fit(data[feature_names], data[target_names])
    data_test = pd.DataFrame(
        np.arange(data.iloc[:, 0].min(), data.iloc[:, 0].max()),
        columns=data[feature_names].columns,
    )
    target_predicted = model.predict(data_test)

    sns.scatterplot(
        x=data.iloc[:, 0], y=data[target_names], color="black", alpha=0.5)
    plt.plot(data_test.iloc[:, 0], target_predicted, linewidth=4)

# Efecto del parámetro max_depth
- El hiperparámetro `max_depth` controla la complejidad general de un árbol de decisión. 
- Este hiperparámetro permite obtener una compensación entre un árbol de decisión subajustado y sobreajustado. 

Construiremos un árbol poco profundo y luego un árbol más profundo, tanto para la clasificación como para la regresión, para comprender el impacto del parámetro.

In [ ]:
# podemos establecer el valor del parámetro max_depth en un valor muy bajo.

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

max_depth = 2
tree_clf = DecisionTreeClassifier(max_depth=max_depth)
tree_reg = DecisionTreeRegressor(max_depth=max_depth)

fit_and_plot_classification(
    tree_clf, data_clf, data_clf_columns, target_clf_column)
_ = plt.title(f"Árbol de clasificación poco profundo con max-depth de {max_depth}")


In [ ]:
fit_and_plot_regression(
    tree_reg, data_reg, data_reg_columns, target_reg_column)
_ = plt.title(f"Árbol de regresión poco profundo con max-depth de {max_depth}")

In [ ]:
# aumentamos el valor del parámetro max_depth para verificar la diferencia observando la función de decisión.

max_depth = 30
tree_clf = DecisionTreeClassifier(max_depth=max_depth)
tree_reg = DecisionTreeRegressor(max_depth=max_depth)

fit_and_plot_classification(
    tree_clf, data_clf, data_clf_columns, target_clf_column)
_ = plt.title(f"Árbol de clasificación profunda con max-depth de {max_depth}")

In [ ]:
fit_and_plot_regression(
    tree_reg, data_reg, data_reg_columns, target_reg_column)
_ = plt.title(f"Árbol de regresión profunda con max-depth de {max_depth}")

> Para las configuración de clasificación y regresión, observamos que aumentar la profundidad hará que el modelo de árbol sea más expresivo.
- Sin embargo, un árbol que es demasiado profundo sobreajustará los datos de entrenamiento, creando particiones que solo son correctas para "valores atípicos (muestras ruidosas)".

`max_depth` es uno de los hiperparámetros que se debe optimizar a través de la validación cruzada y la búsqueda de cuadrícula.**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {"max_depth": np.arange(2, 10, 1)}
tree_clf = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid)
tree_reg = GridSearchCV(DecisionTreeRegressor(), param_grid=param_grid)

In [ ]:
fit_and_plot_classification(
    tree_clf, data_clf, data_clf_columns, target_clf_column)
_ = plt.title(f"Profundidad óptima encontrada a través de CV: "
              f"{tree_clf.best_params_['max_depth']}")

In [ ]:
fit_and_plot_regression(
    tree_reg, data_reg, data_reg_columns, target_reg_column)
_ = plt.title(f"Profundidad óptima encontrada a través de CV: "
              f"{tree_reg.best_params_['max_depth']}")

# Otros hiperparámetros en los árboles de decisión
- El hiperparámetro `max_depth` controla la complejidad general del árbol.
- Este parámetro es adecuado bajo el supuesto de que un árbol se construye simétricamente. 
- Sin embargo, **no hay garantía de que un árbol sea simétrico**.
    - De hecho, se podría alcanzar un rendimiento de generalización óptimo al ahce crecer algunas ramas a más profundidad que otras.

Construiremos un conjunto de datos donde ilustraremos esta asimetría.
- Generaremos un conjunto de datos compuesto por 2 subconjuntos: 
    - un subconjunto donde el árbol debe encontrar una separación clara 
    - y otro subconjunto donde se mezclarán muestras de ambas clases.
- Implica que un árbol de decisión necesitará más divisiones para clasificar las muestras adecuadamente en el segundo subconjunto respecto al primer subconjunto.

In [ ]:
from sklearn.datasets import make_blobs

data_clf_columns = ["Feature #0", "Feature #1"]
target_clf_column = "Class"

# Blobs que serán entrelazadas
X_1, y_1 = make_blobs(
    n_samples=300, centers=[[0, 0], [-1, -1]], random_state=0)
# Blobs que se separarán fácilmente
X_2, y_2 = make_blobs(
    n_samples=300, centers=[[3, 6], [7, 0]], random_state=0)

X = np.concatenate([X_1, X_2], axis=0)
y = np.concatenate([y_1, y_2])
data_clf = np.concatenate([X, y[:, np.newaxis]], axis=1)
data_clf = pd.DataFrame(
    data_clf, columns=data_clf_columns + [target_clf_column])
data_clf[target_clf_column] = data_clf[target_clf_column].astype(np.int32)

In [ ]:
sns.scatterplot(data=data_clf, x=data_clf_columns[0], y=data_clf_columns[1],
                hue=target_clf_column, palette=["tab:red", "tab:blue"])
_ = plt.title("Dataset sintético")

In [ ]:
# entrenaremos un árbol de decisión poco profundo con max_depth = 2.
# Esperaríamos que esta profundidad sea suficiente para separar los blobs que son fáciles de separar.

max_depth = 2
tree_clf = DecisionTreeClassifier(max_depth=max_depth)
fit_and_plot_classification(
    tree_clf, data_clf, data_clf_columns, target_clf_column)
_ = plt.title(f"Árbol de decisión con max-depth de {max_depth}")

In [ ]:
# la representación del árbol.

from sklearn.tree import plot_tree

_, ax = plt.subplots(figsize=(10, 10))
_ = plot_tree(tree_clf, ax=ax, feature_names=data_clf_columns)

In [ ]:
# Vemos que la rama correcta logra una clasificación perfecta. 
# Ahora, aumentamos la profundidad para verificar cómo crecerá el árbol.

max_depth = 6
tree_clf = DecisionTreeClassifier(max_depth=max_depth)
fit_and_plot_classification(
    tree_clf, data_clf, data_clf_columns, target_clf_column)
_ = plt.title(f"Árbol de decisión con max-depth de {max_depth}")

In [ ]:
_, ax = plt.subplots(figsize=(11, 7))
_ = plot_tree(tree_clf, ax=ax, feature_names=data_clf_columns)

> Como se esperaba, la rama izquierda del árbol continúa creciendo, mientras que en la rama derecha no se hacen más divisiones.
- Fijar `max_depth` cortaría el árbol horizontalmente a un nivel específico, ya sea que estofuera más beneficioso o no.

<br>

Los hiperparámetros **`min_samples_leaf`, `min_samples_split`, `max_leaf_nodes` o `min_impurity_decrease`** permiten hacer crecer árboles asimétricos y aplicar una restricción en el nivel de hojas o nodos.

In [ ]:
min_samples_leaf = 60
tree_clf = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
fit_and_plot_classification(
    tree_clf, data_clf, data_clf_columns, target_clf_column)
_ = plt.title(
    f"Árbol de decisión con hoja al menos de {min_samples_leaf} muestras")

In [ ]:
_, ax = plt.subplots(figsize=(10, 7))
_ = plot_tree(tree_clf, ax=ax, feature_names=data_clf_columns)